Lo primero es generar un archivo que genera la app

In [30]:
!pip install plotly

In [1]:
%%writefile app.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
st.title("Frontend")


df = pd.read_pickle("df_stream.pkl")
# Tabla
st.table(df.head())

df1  = df[df["ID"]==1]
df_max_cycles = df.groupby("ID").max("Cycle").reset_index()["Cycle"]
st.bar_chart(df_max_cycles)

#ID engine
st.subheader("Operational settings and sensor variation per engine")
st.text("ID engine must be entered")

engine_option = df["ID"].unique().tolist()
ids_ = st.selectbox("Which engine unit to display?", engine_option,0)
df_id=df[df["ID"]==ids_]

filtered = st.multiselect("Filter columns", options=list(df.columns), default=list(df.columns))
df_id_fil = df_id[filtered]

fig1 = px.line(df_id_fil)
st.write(fig1)

# lr = LinearRegression()
# X_train = df.drop(columns=["ID","Cycle","RUL"])
# y_train = df["RUL"]


# lr.fit(X_train,y_train)
# preds = lr.predict(X_train)

st.title("Remaining useful life prediction")
st.subheader("You're the maintenance chief engineer for airline XXX, you're working in the hangar and an aircraft enters \
        with engine X just performed flight n, what's the predicted number of flights remaining if everything's as usual?")

# st.text("Choose engine number")
# engine_option_ = df["ID"].unique().tolist()
# ids_ = st.selectbox("Which engine unit to display?", engine_option_,1)
# df_id_=df[df["ID"]==ids_]

#st.text("Choose number of flights already performed")
# cycle_option = df_id_["Cycle"].unique().tolist()
# cycles = st.selectbox("How many flights the engine has performed?", cycle_option,0)
# df_id_cycle=df_id_[df_id_["Cycle"]==cycles]

# st.write(df_id_cycle["RUL"])
# filtered = st.multiselect("Filter columns", options=list(df.columns), default=list(df.columns))
# df_id_fil = df_id[filtered]

# fig1 = px.line(preds)
# st.write(fig1)




df_test = pd.read_pickle("df_stream_test.pkl")
rul = pd.read_pickle("df_stream_rul.pkl")

X_train = df.drop(columns=["ID","Cycle","RUL"])
y_train = df["RUL"].clip(upper=125)
X_test = df_test.groupby("ID").last().reset_index().drop(columns=["ID","Cycle"])
y_test = rul

lr = LinearRegression()
lr.fit(X_train, y_train)

st.text("Choose engine number")
engine_option_ = df_test["ID"].unique().tolist()
id_ = st.selectbox("Which engine unit to display?", engine_option_,1)
df_id_=df_test[df_test["ID"]==id_]

cycle_option = df_id_["Cycle"].unique().tolist()
cycles = st.selectbox("How many flights the engine has performed?", cycle_option,2)


# cycles = st.text_input('Number of flights performed')
df_test_cycle = df_test[df_test["Cycle"]==int(cycles)]
X_test_cycle = df_test_cycle.drop(columns=["ID","Cycle"]).reset_index(drop=True)

sensors = ["sensor_measurement_2","sensor_measurement_3","sensor_measurement_4"]

X_affected = X_test_cycle.copy()
percents = np.arange(1,1.1,0.01).tolist()
affect = st.selectbox("Sensors measurement augmented %", percents,2)

for sensor in sensors:
    X_affected[sensor] = X_affected[sensor]*affect
    
preds = lr.predict(X_test_cycle).round(0)[id_]
st.write("theorically, engine shall perform these cycles more: ", preds)

preds_affected = lr.predict(X_affected).round(0)[id_]

if preds_affected <=0:
    st.write("This engine needs maintenance now!")
elif 0 < preds_affected < 20:
    st.write("With these issues, engine is expected to perform these cycles more: ", preds_affected,
            "maintenance shall be schechuled soon")
else:
    st.write("With these issues, engine is expected to perform these cycles more: ", preds_affected,
            "engine should perform good so far")

Overwriting app.py


In [2]:
!streamlit run app.py

^C
